# 基于路由的检索查询

适合于一个数据源，多种检索索引，比如摘要索引和向量索引，或者再加上关键字索引。

让llm根据查询提示词，选择具体某个检索器获取上下文，并生成回答。

本文使用景点数据，效果并不理想，只是能走通流程，并验证确实有效的做了选择。


## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size = 128
Settings.chunk_overlap = 10

CPU times: user 3.52 s, sys: 340 ms, total: 3.86 s
Wall time: 3.51 s


## 加载数据

In [2]:
%%time

items=[
    "颐和园",
    "北海公园",
    "故宫",
]

from llama_index.readers.web import TrafilaturaWebReader

documents = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents)

CPU times: user 737 ms, sys: 22.8 ms, total: 760 ms
Wall time: 1.05 s


3

In [3]:
%%time

nodes = Settings.node_parser.get_nodes_from_documents(documents)

CPU times: user 83.8 ms, sys: 0 ns, total: 83.8 ms
Wall time: 83.1 ms


## 定义摘要索引

In [5]:
%%time

from llama_index.core import SummaryIndex

summary_index = SummaryIndex(nodes)

CPU times: user 44.9 ms, sys: 74 µs, total: 45 ms
Wall time: 44.5 ms


## 定义向量索引

In [7]:
%%time

from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)

CPU times: user 2.44 s, sys: 120 ms, total: 2.56 s
Wall time: 54.6 s


## 定义查询引擎

In [8]:
%%time

from llama_index.core.tools import QueryEngineTool

list_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)
vector_query_engine = vector_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description="适用于检索景点概要信息。",
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "适用于检索景点具体信息片段，例如"
        " 颐和园门票价格等等。"
    ),
)

CPU times: user 22.8 ms, sys: 12 ms, total: 34.8 ms
Wall time: 34.2 ms


## 定义路由查询引擎

In [9]:
%%time

from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    [list_tool, vector_tool],
    index_cls=VectorStoreIndex,
)

CPU times: user 12.6 ms, sys: 330 µs, total: 12.9 ms
Wall time: 1.25 s


In [10]:
%%time


from llama_index.core.query_engine import ToolRetrieverRouterQueryEngine

query_engine = ToolRetrieverRouterQueryEngine(obj_index.as_retriever())

CPU times: user 117 µs, sys: 10 µs, total: 127 µs
Wall time: 130 µs


In [13]:
%%time

response = query_engine.query("介绍下颐和园")
response.response

CPU times: user 53.8 ms, sys: 3.18 ms, total: 57 ms
Wall time: 7.75 s


'颐和园是中国清朝时期的皇家园林，位于北京市西郊海淀区新建宫门路19号。它最初名为清漪园，始建于乾隆十五年（1750年），但在咸丰十年（1860年）被英法联军焚毁后得以重建。颐和园以其独特的布局和设计而闻名，利用了昆明湖和万寿山作为基础，并借鉴了杭州西湖的风景以及江南园林的设计手法和意境。这座大型天然山水园被誉为“皇家园林博物馆”，是保存最为完好的一座皇家行宫御苑。'

In [14]:
%%time

response = query_engine.query("颐和园的门票价格？")
response.response

CPU times: user 58.8 ms, sys: 370 µs, total: 59.1 ms
Wall time: 2.16 s


'颐和园的门票价格在旺季是30元/张，而在淡季则是20元/张。'